# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [1]:
pip install sweetviz

     |████████████████████████████████| 15.1 MB 2.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


## Загрузка и подготовка данных

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import sweetviz as sv
import numpy as np
from IPython.display import display, HTML
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer

In [3]:
warnings.filterwarnings('ignore')

In [4]:
try:
    data_g0 = pd.read_csv('/datasets/geo_data_0.csv')
    data_g1 = pd.read_csv('/datasets/geo_data_1.csv')
    data_g2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    data_g0 = pd.read_csv('geo_data_0.csv')
    data_g1 = pd.read_csv('geo_data_1.csv')
    data_g2 = pd.read_csv('geo_data_2.csv')

### Общий отчет о датасетах

In [5]:
report_g0 = sv.analyze([data_g0, "Location_0"])
report_g0.show_html('common_analysis0.html')

                                             |          | [  0%]   00:00 -> (? left)

Report common_analysis0.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [6]:
report_g1 = sv.analyze([data_g1, "Location_1"])
report_g1.show_html('common_analysis1.html')

                                             |          | [  0%]   00:00 -> (? left)

Report common_analysis1.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [7]:
report_g2 = sv.analyze([data_g2, "Location_2"])
report_g2.show_html('common_analysis2.html')

                                             |          | [  0%]   00:00 -> (? left)

Report common_analysis2.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [8]:
display(HTML('''<script type="text/javascript">
                   window.open("common_analysis0.html", "_blank");
               </script>'''))

In [9]:
display(HTML('''<script type="text/javascript">
                   window.open("common_analysis1.html", "_blank");
               </script>'''))

In [10]:
display(HTML('''<script type="text/javascript">
                   window.open("common_analysis2.html", "_blank");
               </script>'''))

**Вывод: Загружены необходимые библиотеки для работы. Для просмотра всех основных параметров датасетов использована библиотека sweetviz, с помощью которой можно получить отчет о датасетах. Вывести его можно запустив последние 3 ячейки кода выше. По 1 датасету можно сказать, что 2 из 4 признаков распределены нормально, а вот f0 и f1 имеют другую структуру, дубликатов и пропусков нету. Во 2 датасете нормально распределены данные у признака f1, дубликатов и пропусков также нету. В 3 датасете все признаки имеют нормальное распределение, дубликатов и пропусков нету.**

### Подготовка данных

In [11]:
data_g0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [12]:
data_g1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [13]:
data_g0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [14]:
scaler = StandardScaler()
data_g0.loc[:, ['f0_0', 'f1_1', 'f2_2']] = scaler.fit_transform(data_g0.set_index('id').drop(columns='product'))
data_g1.loc[:, ['f0_0', 'f1_1', 'f2_2']] = scaler.transform(data_g1.set_index('id').drop(columns='product'))
data_g2.loc[:, ['f0_0', 'f1_1', 'f2_2']] = scaler.transform(data_g2.set_index('id').drop(columns='product'))

In [15]:
data_g0.head()

,id,f0,f1,f2,product,f0_0,f1_1,f2_2
0,txEyH,0.705745,-0.497823,1.221170,105.280062,0.235512,-1.482793,-0.394515
1,2acmU,1.334711,-0.340164,4.365080,73.037750,0.956946,-1.170246,0.573368
2,409Wp,1.022732,0.151990,1.419926,85.265647,0.599101,-0.194581,-0.333326
3,iJLyR,-0.032172,0.139033,2.978566,168.620776,-0.610890,-0.220269,0.146516
4,Xdl7t,1.988431,0.155413,4.751769,154.036647,1.706773,-0.187797,0.692414


In [16]:
data_g1.head()

,id,f0,f1,f2,product,f0_0,f1_1,f2_2
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103,-17.780771,-16.902525,-0.772273
1,62mP7,14.272088,-3.475083,0.999183,26.953261,15.796320,-7.385019,-0.462856
2,vyE1P,6.263187,-5.948386,5.001160,134.766305,6.609985,-12.288181,0.769192
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408,-15.578326,-23.305902,0.768655
4,AHL4O,12.702195,-8.147433,5.004363,134.766305,13.995628,-16.647650,0.770178


In [17]:
data_g2.head()

,id,f0,f1,f2,product,f0_0,f1_1,f2_2
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673,-1.889601,1.413843,-1.025669
1,WJtFt,0.262778,0.269839,-2.530187,56.069697,-0.272578,0.039046,-1.549407
2,ovLUW,0.194587,0.289035,-5.586433,62.871910,-0.350794,0.077101,-2.490303
3,q6cA6,2.236060,-0.553760,0.930038,114.572842,1.990807,-1.593685,-0.484143
4,WPMUX,-0.515993,1.716266,5.899011,149.600746,-1.165841,2.906493,1.045604


## Обучение и проверка модели

In [18]:
# Функция для обучения и оценки модели на каждом регионе
def train_and_evaluate(data):
    # Разделение данных на признаки (X) и целевой признак (y)
    X = data[['f0_0', 'f1_1', 'f2_2']]
    y = data['product']
    
    # Разделение на обучающую и валидационную выборки
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)
    
    # Обучение модели линейной регрессии
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Предсказание на валидационной выборке
    y_pred = model.predict(X_valid)
    
    # Сохранение предсказаний и фактических значений
    predictions = pd.DataFrame({'actual': y_valid, 'predicted': y_pred})
    
    # Вычисление среднего запаса предсказанного сырья
    mean_predicted = np.mean(y_pred)
    
    # Вычисление RMSE
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    
    # Печать результатов
    print(f"Средний запас предсказанного сырья: {mean_predicted:.2f} тыс. баррелей")
    print(f"RMSE модели: {rmse:.2f}")
    
    return predictions

In [19]:
# Обучение и оценка для каждого региона
print("Регион 0:")
predictions_0 = train_and_evaluate(data_g0)

print("\nРегион 1:")
predictions_1 = train_and_evaluate(data_g1)

print("\nРегион 2:")
predictions_2 = train_and_evaluate(data_g2)

Регион 0:
Средний запас предсказанного сырья: 92.40 тыс. баррелей
RMSE модели: 37.76

Регион 1:
Средний запас предсказанного сырья: 68.71 тыс. баррелей
RMSE модели: 0.89

Регион 2:
Средний запас предсказанного сырья: 94.77 тыс. баррелей
RMSE модели: 40.15


**Вывод: На данном шаге написана функция, которая обучает модель Линейной регрессии, делает предсказания на валидационной выборке и расчитывает метрику RMSE. Получаем, что модель для 0 региона показывает один из наибольших средних запасов сырья, но rmse велика, для 1 региона наименьший запас сырья, но rmse имеет маленькое значение, что очень хорошо. Для 2 региона параметры почти такие же как и для региона 2, но средний запас чуть больше и rmse тоже увеличилась.**

## Подготовка к расчёту прибыли

In [20]:
ALL_PLACE = 500 # Всего выбирают 500 точек
LEARN_PLACE = 200 # с помощью машинного обучения
ALL_MONEY = 10 ** 10 # Общий бюджет
PROFIT_ONE_BARREL= 450 # Цена за один баррель
PROFIT_ONE_PRODUCT= 45 * (10 **4) # Цена с единицы продукта (Объем в тысячах баррелей)

In [21]:
# Расчет достаточного объема сырья для безубыточности
sufficient_volume = ALL_MONEY / (PROFIT_ONE_PRODUCT * LEARN_PLACE)
print(f"Достаточный объем сырья для безубыточности: {sufficient_volume:.2f} тыс. баррелей")

# Средние запасы предсказанного сырья по регионам
avg_reserves_region_012 = [92.4, 68.71, 94.77]
region = [0, 1, 2]
for reserve, region in zip(avg_reserves_region_012, region):
    print(f"Средний запас в Регионе {region}: {reserve} тыс. баррелей")
    if reserve > sufficient_volume:
        print(f"В регионе {region} средний запас сырья больше, чем объем сырья необходимый для безубыточности.")
    else: 
        print(f"В регионе {region} средний запас сырья меньше, чем объем сырья необходимый для безубыточности.")
    print("+----------------------------------+")

Достаточный объем сырья для безубыточности: 111.11 тыс. баррелей
Средний запас в Регионе 0: 92.4 тыс. баррелей
В регионе 0 средний запас сырья меньше, чем объем сырья необходимый для безубыточности.
+----------------------------------+
Средний запас в Регионе 1: 68.71 тыс. баррелей
В регионе 1 средний запас сырья меньше, чем объем сырья необходимый для безубыточности.
+----------------------------------+
Средний запас в Регионе 2: 94.77 тыс. баррелей
В регионе 2 средний запас сырья меньше, чем объем сырья необходимый для безубыточности.
+----------------------------------+


**Вывод: Получаем, что для того, чтобы постройка скважин окупилась, их необходимая производительность должна быть 111 тысяч баррелей. Но однако среднего запаса сырья предсказанного моделью не хватает для того, чтобы скважины окупились во всех регионах.** 

## Расчёт прибыли и рисков 

In [22]:
def calculate_profit(predictions, true_values, count_wells, price_per_product):
    selected_indices = np.argsort(predictions)[-count_wells:]  # Индексы лучших скважин по предсказаниям
    selected_true_values = true_values.iloc[selected_indices]  # Соответствующие истинные значения
    total_volume = selected_true_values.sum()
    profit = (total_volume * price_per_product - ALL_MONEY) / (10 ** 9)
    return profit

In [23]:
# Параметры
count_wells = 200  # Количество лучших скважин для расчета прибыли
# Расчет прибыли
profit_0 = calculate_profit(predictions_0['predicted'], predictions_0['actual'], count_wells, PROFIT_ONE_PRODUCT)
print(f"Прибыль для района 0: {profit_0:.2f} млрд. руб.")
profit_1 = calculate_profit(predictions_1['predicted'], predictions_1['actual'], count_wells, PROFIT_ONE_PRODUCT)
print(f"Прибыль для района 1: {profit_1:.2f} млрд. руб.")
profit_2 = calculate_profit(predictions_2['predicted'], predictions_2['actual'], count_wells, PROFIT_ONE_PRODUCT)
print(f"Прибыль для района 2: {profit_2:.2f} млрд. руб.")

Прибыль для района 0: 3.36 млрд. руб.
Прибыль для района 1: 2.42 млрд. руб.
Прибыль для района 2: 2.60 млрд. руб.


**Вывод: Регион 1 показывает лучший результат, скважины в том районе приносят наибольшую прибыль.**

In [24]:
state = np.random.RandomState(12345)
def bootstrap_profit(data, count_wells, price_per_product):
    n_samples=1000
    profits = []
    for _ in range(n_samples):
        subsample = data.sample(n=ALL_PLACE, replace=True, random_state=state)
        selected_data = subsample.sort_values(by='predicted', ascending=False).head(count_wells)
        profit = calculate_profit(selected_data['predicted'], selected_data['actual'], count_wells, price_per_product)
        profits.append(profit)
    profits = np.array(profits)
    mean_profit = np.mean(profits)
    lower = np.percentile(profits, 2.5)
    upper = np.percentile(profits, 97.5)
    risk_of_loss = (profits < 0).mean()  # Доля выборок с убытками
    
    return mean_profit, (lower, upper), risk_of_loss


In [25]:
# Расчет прибыли и рисков для региона 0
mean_profit_0, confidence_interval_0, risk_0 = bootstrap_profit(predictions_0, count_wells, PROFIT_ONE_PRODUCT)
print(f"Регион 0: Средняя прибыль = {mean_profit_0:.2f} млрд. руб., 95% доверительный интервал = {confidence_interval_0}, Риск убытков = {risk_0:.2%}")

# Расчет прибыли и рисков для региона 1
mean_profit_1, confidence_interval_1, risk_1 = bootstrap_profit(predictions_1, count_wells, PROFIT_ONE_PRODUCT)
print(f"Регион 1: Средняя прибыль = {mean_profit_1:.2f} млрд. руб., 95% доверительный интервал = {confidence_interval_1}, Риск убытков = {risk_1:.2%}")

# Расчет прибыли и рисков для региона 2
mean_profit_2, confidence_interval_2, risk_2 = bootstrap_profit(predictions_2, count_wells, PROFIT_ONE_PRODUCT)
print(f"Регион 2: Средняя прибыль = {mean_profit_2:.2f} млрд. руб., 95% доверительный интервал = {confidence_interval_2}, Риск убытков = {risk_2:.2%}")

Регион 0: Средняя прибыль = 0.41 млрд. руб., 95% доверительный интервал = (-0.11774213649487013, 0.9117370507514055), Риск убытков = 6.70%
Регион 1: Средняя прибыль = 0.44 млрд. руб., 95% доверительный интервал = (0.03572848928085324, 0.8280066390043884), Риск убытков = 1.60%
Регион 2: Средняя прибыль = 0.39 млрд. руб., 95% доверительный интервал = (-0.16478516610904617, 0.8882062341976783), Риск убытков = 7.80%


Регион 1 показывает наивысшую среднюю прибыль среди всех трех регионов. Однако все значения достаточно близки, что указывает на сравнительно небольшую разницу в ожидаемой доходности между регионами.


Для регионов 0 и 2 нижняя граница доверительного интервала отрицательная, что указывает на вероятность того, что прибыль может оказаться отрицательной. В регионе 1 обе границы положительные, что делает этот регион более стабильным с точки зрения предсказанной прибыли.

Регион 1 имеет наименьший риск убытков (1.6%), что делает его наиболее безопасным вариантом. В то время как регионы 0 и 2 имеют более высокий риск убытков (6.7% и 7.8% соответственно).


## Итоговый вывод

В ходе выполнения проекта были выполнены следующие шаги:

- Подключены необходимые инструменты для работы. С помощью sweetviz выведена общая информация о датасетах и распределение данных.

- Далее построена функция, которая обучает модель Линейной регрессии, делает предсказания на валидационной выборке и расчитывает метрику RMSE. Получаем, что модель для 0 региона показывает один из наибольших средних запасов сырья, но rmse велика, для 1 региона наименьший запас сырья, но rmse имеет маленькое значение, что очень хорошо. Для 2 региона параметры почти такие же как и для региона 2, но средний запас чуть больше и rmse тоже увеличилась.

- После этого посчитано количество сырья необходимое для того, чтобы постройка скважин окупилась - 111 тыс. баррелей. Также посчитано сколько сможет дать прибыли скважины в каждом регионе и применена техника bootstrap для оценки распределения прибыли, полуены следующие результаты:

    1. **Средняя прибыль**:
       - Регион 0: Средняя прибыль составляет 0.41 млрд руб.
       - Регион 1: Средняя прибыль составляет 0.44 млрд руб.
       - Регион 2: Средняя прибыль составляет 0.39 млрд руб.
   
       Регион 0 имеет самую высокую среднюю прибыль среди всех трех регионов, что делает его наиболее привлекательным с точки зрения ожидаемой доходности.

    2. **95%-й доверительный интервал**:
       - Регион 0: (-0.118, 0.912) млрд руб.
       - Регион 1: (0.036, 0.828) млрд руб.
       - Регион 2: (-0.165, 0.888) млрд руб.

       - Для регионов 0 и 2 нижняя граница доверительного интервала отрицательная, что указывает на вероятность того, что прибыль может оказаться отрицательной. 
       - В регионе 1 обе границы положительные, что делает этот регион более стабильным с точки зрения предсказанной прибыли.

    3. **Риск убытков**:
       - Регион 0: Риск убытков составляет 6.7%.
       - Регион 1: Риск убытков составляет 1.6%.
       - Регион 2: Риск убытков составляет 7.8%.

       
1. Регион 1 является наиболее предпочтительным для разработки скважин, поскольку:
   - Он показывает наивысшую среднюю прибыль (0.44 млрд руб.).
   - Он единственный, где доверительный интервал полностью положительный.
   - У него самый низкий риск убытков (1.6%).

2. Регион 0 и Регион 2 менее привлекательны:
   - Оба имеют отрицательные границы доверительных интервалов, что указывает на вероятность убытков.
   - Риск убытков выше по сравнению с регионом 1.

Таким образом, Регион 1 выглядит наиболее безопасным и прибыльным для инвестиций, несмотря на сравнительно невысокую среднюю прибыль.